In [15]:
import numpy as np
import pandas as pd
import GPy

In [16]:
customers = pd.read_csv('./customers.csv')
transactions = pd.read_csv('./database.csv')

In [17]:
customers.head()

,Unnamed: 0,account_id,district_id,frequency,date,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,0,576,55,0,1993-01-01,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
1,1,704,55,0,1993-01-01,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
2,2,192,55,0,1993-01-08,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
3,3,10364,55,0,1993-01-17,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
4,4,497,55,0,1993-04-15,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894


In [18]:
transactions.head()

,Unnamed: 0,account_id,trans_id,category,k-symbol,date,amount,mean_income,Initial Balance
0,0,1,5,0,-1,1995-03-24,122.633333,1358.788889,33.333333
1,1,1,199,29,-1,1995-04-13,420.000000,1358.788889,33.333333
2,2,1,3530438,40,2,1995-04-23,0.640000,1358.788889,33.333333
3,3,1,6,0,-1,1995-04-30,122.633333,1358.788889,33.333333
4,4,1,200,46,-1,1995-05-13,70.000000,1358.788889,33.333333


In [19]:
data = pd.merge(customers, transactions, on='account_id')[['account_id', 'category','amount', 'mean_income', 'A4', 'A11', 'A13', 'A16']]

In [20]:
data['A16'] = data['A16']/data['A4']

In [21]:
data = data[data['amount']>0]

In [22]:
strats = []
for cat in data['category'].unique():
    strats.append((cat, data[data['category']==cat].sample(n=200, replace=True)))

In [23]:
X = []
Y = []
for _, datum in strats:
    X.append(datum[['mean_income','A4','A11','A13','A16']].as_matrix())
    Y.append(datum['amount'].as_matrix()[:,None])

In [24]:
models = []
for XX, YY in zip(X,Y):
    K = GPy.kern.RBF(5, ARD=True)
    m = GPy.models.GPRegression(XX,YY,K)
    m.optimize()
    models.append(m)

In [25]:
test_strats = []
for cat in data['category'].unique():
    test_strats.append((cat, data[data['category']==cat].sample(n=200, replace=True)))

In [26]:
test_X = []
for _, datum in strats:
    test_X.append(datum[['mean_income','A4','A11','A13','A16']].as_matrix())

In [27]:
test_Y = []
for m, XX in zip(models, test_X):
    test_Y.append(m.posterior_samples(XX,size=1))

 /home/ayman/anaconda3/envs/ati-dsg/lib/python3.6/site-packages/GPy/core/gp.py:498: RuntimeWarning:covariance is not positive-semidefinite.


In [33]:
categories = []
for cat, _ in strats:
    for i in range(200):
        categories.append(cat)

categories=np.vstack(categories)

In [36]:
amounts = np.vstack(test_Y)
features = np.vstack(test_X)

In [37]:
synthetic_data = np.hstack([features, categories, amounts])

In [38]:
synthetic_data.shape

(10000, 7)

In [41]:
synthetic_data=pd.DataFrame(synthetic_data)

In [42]:
synthetic_data.columns=[['mean_income','A4','A11','A13','A16', 'category', 'amount']]

In [43]:
synthetic_data.head()

,mean_income,A4,A11,A13,A16,category,amount
0,12928.861905,285387.0,10177.0,7.75,0.035419,6.0,215.487129
1,3157.512030,95907.0,8388.0,2.94,0.017392,6.0,212.587635
2,2214.243655,75232.0,8980.0,2.21,0.037391,6.0,221.026832
3,2864.307692,75637.0,9622.0,0.59,0.046657,6.0,192.982195
4,2201.057500,77963.0,8546.0,4.02,0.040019,6.0,200.140121


In [44]:
synthetic_data.to_csv('./example_synthetic_data.csv')